In [ ]:
pip install requests pandas schedule

### This code saves data in different csv in order to maintain accuracy.

In [ ]:
import requests
import pandas as pd
import schedule
import time
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
# API Configuration
API_KEY = 'f1cbffee79830f24744dab0edfb64167143f5102'
BASE_URL = 'https://api.jcdecaux.com/vls/v3'

def fetch_bike_data(contract_name='dublin'):
    """Fetch bike station data for a specified contract."""
    url = f"{BASE_URL}/stations?contract={contract_name}&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        
        # Format lastUpdate to readable format
        df['lastUpdate'] = pd.to_datetime(df['lastUpdate'])
        df['lastUpdate'] = df['lastUpdate'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Generate a filename with a timestamp to ensure uniqueness
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"Dublin Bikes Dat a_{timestamp}.csv"
        
        # Save the dataframe to a CSV file
        df.to_csv(filename, index=False)
        print(f"Data fetched and saved at {datetime.now()} in file {filename}")
    else:
        print(f"Failed to fetch data: {response.status_code}")

# Set up a schedule to run the data collection every 20 minutes
schedule.every(30).minutes.do(fetch_bike_data, contract_name='dublin')

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(60)

In [ ]:
import os
print("Current Working Directory:", os.getcwd())

### All the csv file obtained were combine for further analysis

In [ ]:
import os
import pandas as pd

def merge_all_csv_files_in_folder(folder_path, output_file):
    """
    Merges all CSV files in a folder into a single DataFrame and saves the result to a file.
    :param folder_path: Folder path containing the CSV files
    :param output_file: File path to save the merged CSV
    """
    # Get all CSV files from the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    if not csv_files:
        print("No CSV files found in the directory!")
        return None

    print(f"Found {len(csv_files)} files to merge.")
    
    # List to hold DataFrames
    dataframes = []

    # Loop through and read each CSV file
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        print(f"Processing: {csv_file}")
        df = pd.read_csv(file_path)
        dataframes.append(df)

    # Merge all DataFrames
    merged_df = pd.concat(dataframes, ignore_index=True)
    
    # Drop duplicates if necessary
    print(f"Initial rows: {merged_df.shape[0]}")
    merged_df.drop_duplicates(inplace=True)
    print(f"Rows after removing duplicates: {merged_df.shape[0]}")

    # Save merged DataFrame to CSV
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data saved to: {output_file}")

    return merged_df

# Specify folder path and output file
folder_path = r"C:\Users\raksh\Dublin_bikes_data"  # Replace with your folder path
output_file = r"C:\Users\raksh\merged_bike_data1.csv"  # Replace with desired output file path

# Merge all CSV files
merged_data = merge_all_csv_files_in_folder(folder_path, output_file)

# Display a preview of the merged data
if merged_data is not None:
    print(merged_data.head())
